In [1]:
import pandas as pd
from ete3 import NCBITaxa
import os
import math
import re
import numpy as np
from Bio import Entrez
import time
import json

Run on an AWS EC2 instance.

cd /mnt/data

In [ ]:
os.system("aws s3 ls s3://czbiohub-mosquito/contigs/ --recursive | "+
          "grep '.blast.' | grep -v 'ater' | awk 'NF>1{print $NF}' | "+
          "parallel aws s3 cp s3://czbiohub-mosquito/{} {}")

In [ ]:
os.system("mkdir contig_lineages")

In [ ]:
ncbi = NCBITaxa()
#ncbi.update_taxonomy_database()

In [ ]:
Entrez.email = "lucy.li@czbiohub.org"

In [2]:
blast_col_names = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", 
                   "qstart", "qend", "sstart", "send", "evalue", "bitscore"]

In [ ]:
blast_results = []
for path, subdirs, files in os.walk("contigs"):
    for name in files:
        fn = os.path.join(path, name)
        tb = pd.read_table(fn, header=None, names=blast_col_names)
        tb = tb.assign(blast_type=name.split('.')[0], sample=os.path.basename(path))
        blast_results.append(tb)
blast_results_df = pd.concat(blast_results)

In [ ]:
blast_topresults_df = blast_results_df.groupby(['sample', 'blast_type', 'qseqid']).first()

In [ ]:
accessions = blast_topresults_df.sseqid.unique()

In [ ]:
def get_taxid (x):
    search_result = Entrez.parse(Entrez.esummary(db="nucleotide", id=x))
    taxid = str(list(search_result)[0]['TaxId'])
    return (taxid)

In [ ]:
acc_to_lineage = {}

In [ ]:
for x in accessions:
    if (x in acc_to_lineage):
        continue
    for i in range(5):
        try:
            taxid = get_taxid(x)
        except:
            time.sleep(20)
        else:
            break
    acc_to_lineage[x] = ncbi.get_lineage(taxid)


In [ ]:
with open('acc_to_lineage.json', 'w') as file:
    json.dump(acc_to_lineage, file)

In [ ]:
# accessions_file = open("contig_lineages/accessions.txt", "w")
# for x in accessions:
#     accessions_file.write(x+"\n")

# accessions_file.close()
# os.system("cat contig_lineages/accessions.txt | parallel -j 1 python get_lineage_acc.py {}")